# 閉包模型鏈 (Closure Model Chains)

本範例展示如何在 RunnableLambda 中通過閉包方式直接調用模型，實現完全自定義的處理邏輯。

## 🎯 學習目標
- 理解閉包在 LangChain 中的應用
- 掌握在自定義函數中調用模型的技巧
- 學習完全控制處理流程的方法

## 📝 核心概念
閉包模型鏈通過在 RunnableLambda 函數內部捕獲外部的模型實例，實現靈活的自定義處理邏輯，包括複雜的條件判斷、多步處理和日誌記錄。

## ✨ 適用場景
- 需要複雜條件判斷的處理流程
- 多步驟自定義處理
- 需要日誌記錄和監控
- 需要完全控制模型調用邏輯

## 步驟 1：導入套件和建立模型

In [ ]:
# 導入必要的套件
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda
from langchain_google_genai import ChatGoogleGenerativeAI

# 載入環境變數
load_dotenv()

# 建立 Ollama 模型
model = ChatGoogleGenerativeAI(model="gemini-flash-2.5")

print("✅ 模型已建立")

## 步驟 2：定義主提示模板

In [ ]:
# 定義主提示模板：生成郵件回覆
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "你是一個專業的客服代表。"),
    ("human", "請回覆這封客戶郵件：\n\n{email_content}")
])

print("✅ 主提示模板已建立")

## 步驟 3：定義閉包函數（在內部調用模型）

這個函數通過閉包捕獲外部的 `model` 變數，在函數內部完成：
1. 格式化郵件
2. 準備品質檢查的 Prompt
3. 調用模型進行品質檢查
4. 組合最終結果

In [ ]:
def format_with_quality_check(reply):
    """
    格式化郵件並使用模型進行品質檢查
    
    這個函數展示了閉包的強大功能：
    - 捕獲外部的 model 變數
    - 在函數內部完全控制處理邏輯
    - 可以添加日誌、條件判斷等自定義邏輯
    """
    
    # 步驟 1：格式化郵件
    formatted_reply = f"""
親愛的客戶，

{reply}

感謝您的來信，如有其他問題請隨時聯繫我們。

此致
客服團隊
"""
    
    # 步驟 2：準備品質檢查的 Prompt
    quality_check_prompt = ChatPromptTemplate.from_messages([
        ("system", "你是一個郵件品質審查專家。請評估這封郵件的專業度、友善度和完整度，並給出1-10分的評分和簡短評語。"),
        ("human", "請評估這封郵件：\n\n{email}")
    ])
    
    # 步驟 3：調用模型進行品質檢查（這裡使用閉包捕獲的 model）
    quality_result = quality_check_prompt.format_prompt(email=formatted_reply)
    quality_score = model.invoke(quality_result.to_string())
    
    # 步驟 4：組合最終結果
    final_result = f"""
{'='*60}
📧 格式化的郵件回覆：
{'='*60}
{formatted_reply}

{'='*60}
📊 AI 品質檢查結果：
{'='*60}
{quality_score}
"""
    return final_result

print("✅ 閉包函數已定義完成")

## 步驟 4：建立包含閉包模型調用的鏈

In [ ]:
# 建立包含模型調用的自訂處理步驟
format_and_check = RunnableLambda(format_with_quality_check)

# 建立完整的鏈
closure_chain = (
    prompt_template         # 1. 主 Prompt
    | model                 # 2. 主模型生成回覆
    | StrOutputParser()     # 3. 解析為字串
    | format_and_check      # 4. 格式化 + 品質檢查（內部調用模型）
)

print("✅ 閉包模型鏈已建立完成！")
print("\n流程：主 Prompt → 主模型 → 格式化 + 品質檢查模型（閉包） → 最終結果")

## 步驟 5：測試閉包模型鏈

In [ ]:
# 準備測試資料
customer_email = """
您好，

我最近購買了貴公司的產品，但是發現包裝有損壞。
請問可以退貨或換貨嗎？

王小明
"""

# 執行閉包鏈
print("="*60)
print("客戶郵件：")
print("="*60)
print(customer_email)

print("\n" + "="*60)
print("閉包模型鏈處理結果：")
print("="*60)

result = closure_chain.invoke({"email_content": customer_email})
print(result)

## 💡 進階範例：添加條件判斷和日誌記錄

閉包模型鏈的優勢在於可以在函數內部添加任何自定義邏輯：

In [ ]:
import datetime

def advanced_format_with_quality_check(reply):
    """
    進階版：添加條件判斷和日誌記錄
    """
    
    # 日誌記錄
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"[{timestamp}] 開始處理郵件回覆...")
    
    # 條件判斷：根據回覆內容選擇不同的格式
    if "退貨" in reply or "換貨" in reply:
        formatted_reply = f"""
親愛的客戶，

{reply}

【退換貨政策】
- 自收貨日起 7 天內可退換貨
- 商品需保持完整包裝
- 請聯繫客服安排退換貨事宜

此致
客服團隊
"""
        check_focus = "請特別注意退換貨政策的說明是否清楚。"
    else:
        formatted_reply = f"""
親愛的客戶，

{reply}

感謝您的來信，如有其他問題請隨時聯繫我們。

此致
客服團隊
"""
        check_focus = "請評估整體的專業度和友善度。"
    
    print(f"[{timestamp}] 郵件已格式化，開始品質檢查...")
    
    # 品質檢查
    quality_check_prompt = ChatPromptTemplate.from_messages([
        ("system", f"你是郵件品質審查專家。{check_focus}"),
        ("human", "評估這封郵件：\n\n{email}")
    ])
    
    quality_result = quality_check_prompt.format_prompt(email=formatted_reply)
    quality_score = model.invoke(quality_result.to_string())
    
    print(f"[{timestamp}] 品質檢查完成！")
    
    # 組合結果
    final_result = f"""
{'='*60}
📧 格式化的郵件回覆：
{'='*60}
{formatted_reply}

{'='*60}
📊 AI 品質檢查結果（檢查重點：{check_focus}）：
{'='*60}
{quality_score}

{'='*60}
⏰ 處理時間：{timestamp}
{'='*60}
"""
    return final_result

# 建立進階版鏈
advanced_chain = (
    prompt_template
    | model
    | StrOutputParser()
    | RunnableLambda(advanced_format_with_quality_check)
)

print("\n" + "="*60)
print("進階閉包模型鏈測試（含條件判斷和日誌）：")
print("="*60)

result = advanced_chain.invoke({"email_content": customer_email})
print(result)

## 📊 閉包模型鏈總結

### ✅ 優點
- **靈活度極高**：可以在函數內部實現任何自定義邏輯
- **完全控制**：完全掌控模型調用的時機和方式
- **易於擴展**：可以輕鬆添加日誌、監控、條件判斷等功能
- **閉包特性**：自動捕獲外部變數，無需顯式傳遞

### ⚠️ 注意事項
- **可讀性較差**：邏輯隱藏在函數內部，不易從鏈的結構看出流程
- **難以追蹤**：調試時較難追蹤模型調用的詳細過程
- **維護成本**：複雜邏輯可能導致函數過於龐大

### 🎯 最佳實踐
1. **適度使用**：只在確實需要複雜邏輯時使用閉包方式
2. **清晰命名**：函數名稱應清楚說明其功能
3. **添加註解**：在函數內部添加詳細註解說明邏輯
4. **日誌記錄**：添加日誌以便追蹤和調試

### 🎯 使用時機
- ✅ 需要複雜的條件判斷
- ✅ 需要多步驟自定義處理
- ✅ 需要日誌記錄和監控
- ❌ 簡單的線性流程（建議使用[串聯模型鏈](5_chains_sequential_model_ollama.ipynb)）
- ❌ 需要高可讀性的場景（建議使用[動態提示鏈](9_chains_dynamic_prompt_ollama.ipynb)）

### 🔗 相關技術
- [串聯模型鏈](5_chains_sequential_model_ollama.ipynb)：簡單的順序調用
- [並行模型鏈](8_chains_parallel_model_ollama.ipynb)：同時調用多個模型
- [動態提示鏈](9_chains_dynamic_prompt_ollama.ipynb)：推薦的最佳實踐